In [34]:
import requests

url = "https://www.base.gov.pt/Base4/pt/resultados/"

headers = {
    "Accept": "text/plain, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en,pt-PT;q=0.9,pt;q=0.8,en-US;q=0.7",
    "Connection": "keep-alive",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.base.gov.pt",
    "Referer": "https://www.base.gov.pt/Base4/pt/pesquisa/?type=entidades&texto=506215695",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "sec-ch-ua": '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"'
}

cookies = {
    "f5avraaaaaaaaaaaaaaaa_session_": "IOHOFFGIOIEMEJBNJJFFJABAJGJLOFMJHLNBFKGPFOIGLFOPHDBKNDCBKIFFCMJGLFCDLCMJNGCHOAIJALNAODOIAKEBFKBDCJDHEJGJIPPDEOFEEDPBHBDBKIGNFAFH",
    "f5_cspm": "1234"
}

data = {
    "type": "search_entidades",
    "version": "108.0",
    "query": "texto=506215695",
    "sort": "+description",
    "page": "0",
    "size": "25"
}

response = requests.post(url, headers=headers, cookies=cookies, data=data)

print("Status code:", response.status_code)
print("First 1000 characters of response:")
print(response.text[:1000])


Status code: 200
First 1000 characters of response:
					{ "total": 1, "items": [{"nif":"506215695","description":"Município do Fundão","id":1129,"country":"Portugal"}] }



In [ ]:
import requests
import math
import csv

def get_total_contracts(adjudicante_id):
    url = "https://www.base.gov.pt/Base4/pt/resultados/"
    headers = {
        "Accept": "text/plain, */*; q=0.01",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://www.base.gov.pt",
        "Referer": f"https://www.base.gov.pt/Base4/pt/pesquisa/?type=contratos&adjudicanteid={adjudicante_id}",
        "User-Agent": "Mozilla/5.0",
        "X-Requested-With": "XMLHttpRequest"
    }
    data = {
        "type": "search_contratos",
        "version": "108.0",
        "query": f"adjudicanteid={adjudicante_id}",
        "page": "0",
        "size": "1"
    }

    response = requests.post(url, headers=headers, data=data)
    response.encoding = "utf-8"  # 💥 Add this line
    response.raise_for_status()
    return response.json().get("total", 0)

def get_contracts_by_adjudicante_id(adjudicante_id):
    url = "https://www.base.gov.pt/Base4/pt/resultados/"
    headers = {
        "Accept": "text/plain, */*; q=0.01",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://www.base.gov.pt",
        "Referer": f"https://www.base.gov.pt/Base4/pt/pesquisa/?type=contratos&adjudicanteid={adjudicante_id}",
        "User-Agent": "Mozilla/5.0",
        "X-Requested-With": "XMLHttpRequest"
    }

    total = get_total_contracts(adjudicante_id)
    page_count = math.ceil(total / 25)
    print(f"📄 Total contracts: {total} → Pages to fetch: {page_count}")

    all_contracts = []

    for page in range(page_count):
        data = {
            "type": "search_contratos",
            "version": "108.0",
            "query": f"adjudicanteid={adjudicante_id}",
            "page": str(page),
            "size": "25"
        }

        response = requests.post(url, headers=headers, data=data)
        response.encoding = "utf-8"  # ← Add this
        if response.status_code != 200:
            print(f"❌ Stopped at page {page} with status {response.status_code}")
            break

        contracts = response.json().get("items", [])
        if not contracts:
            break

        all_contracts.extend(contracts)

    return all_contracts

def save_contracts_to_csv(contracts, filename="./datasets/financial/contracts.csv"):
    if not contracts:
        print("⚠️ No contracts to save.")
        return

    keys = contracts[0].keys()
    with open(filename, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        writer.writerows(contracts)

    print(f"✅ Saved {len(contracts)} contracts to {filename}")

# 🔧 Run
if __name__ == "__main__":
    adjudicante_id = 1129  # Município do Fundão
    contracts = get_contracts_by_adjudicante_id(adjudicante_id)
    print(f"\n✅ Found {len(contracts)} contracts for ID {adjudicante_id}")
    save_contracts_to_csv(contracts)


📄 Total contracts: 2124 → Pages to fetch: 85

✅ Found 2124 contracts for ID 1129
✅ Saved 2124 contracts to contracts.csv


# Get Base.Gov Contracts for a list of NIFs

In [1]:
import requests
import pandas as pd

# List of NIFs you want to search for
nif_list = ["506215695", # Fundão
            "501143530", # Castelo Branco
            "506824152", # Oleiros
            "506192164", # Penamacor
            "501121030", # Idanha a Nova
            "506811662", # Sabugal
            "506695956", # Belmonte
            "505330768", # Covilha
            "506811883", # Pampilhosa da Serra
            ]  # Add more NIFs as needed

url = "https://www.base.gov.pt/Base4/pt/resultados/"

headers_template = {
    "Accept": "text/plain, */*; q=0.01",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.base.gov.pt",
    "User-Agent": "Mozilla/5.0",
    "X-Requested-With": "XMLHttpRequest"
}

# Store results here
results = []

for nif in nif_list:
    headers = headers_template.copy()
    headers["Referer"] = f"https://www.base.gov.pt/Base4/pt/pesquisa/?type=entidades&texto={nif}"

    data = {
        "type": "search_entidades",
        "version": "108.0",
        "query": f"texto={nif}",
        "sort": "+description",
        "page": "0",
        "size": "25"
    }

    try:
        response = requests.post(url, headers=headers, data=data)
        response.encoding = "utf-8"
        response.raise_for_status()
        items = response.json().get("items", [])
        results.extend(items)
    except Exception as e:
        print(f"❌ Error with NIF {nif}: {e}")

# Create a DataFrame
df = pd.DataFrame(results)
print(df)

         nif     id   country                       description
0  506215695   1129  Portugal               Município do Fundão
1  501143530   3477  Portugal       Município de Castelo Branco
2  506824152  34128  Portugal              Município de Oleiros
3  506192164   8781  Portugal            Município de Penamacor
4  501121030    825  Portugal        Município de Idanha-a-Nova
5  506811662   1553  Portugal              Município de Sabugal
6  506695956   3816  Portugal             Município de Belmonte
7  505330768   4567  Portugal              Município da Covilhã
8  506811883   5416  Portugal  Município de Pampilhosa da Serra


In [6]:
import requests
import math
import pandas as pd
import time
import random
import json
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

# Output paths
base_dir = Path("./datasets/base-gov")
logs_dir = base_dir / "logs"
failed_pages_dir = base_dir / "failed_pages"
output_dir = base_dir
completed_log_path = base_dir / "completed_nifs.log"

for d in [logs_dir, failed_pages_dir, output_dir]:
    d.mkdir(parents=True, exist_ok=True)

# Load completed NIFs
completed_nifs = set()
if completed_log_path.exists():
    with open(completed_log_path, "r") as f:
        completed_nifs = set(line.strip() for line in f.readlines())

# Retry logic with exponential backoff
def retry_request(post_func, max_retries=3, backoff=2):
    for attempt in range(max_retries):
        try:
            return post_func()
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(backoff * (attempt + 1))
            else:
                raise e

# Log helper
def log_line(log_fp, message):
    print(message)
    with open(log_fp, "a", encoding="utf-8") as f:
        f.write(message + "\n")

# Main contract scraping function
def download_contracts_for_nif(row):
    adjudicante_id = row["id"]
    nif = str(row["nif"])
    name = row["description"]

    if nif in completed_nifs:
        print(f"⏭ Skipping NIF {nif}, already processed.")
        return

    log_fp = logs_dir / f"{nif}.log"
    failed_fp = failed_pages_dir / f"{nif}_failed.json"

    url = "https://www.base.gov.pt/Base4/pt/resultados/"
    headers = {
        "Accept": "text/plain, */*; q=0.01",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://www.base.gov.pt",
        "Referer": f"https://www.base.gov.pt/Base4/pt/pesquisa/?type=contratos&adjudicanteid={adjudicante_id}",
        "User-Agent": "Mozilla/5.0",
        "X-Requested-With": "XMLHttpRequest"
    }

    def get_total():
        def post():
            data = {"type": "search_contratos", "version": "108.0",
                    "query": f"adjudicanteid={adjudicante_id}", "page": "0", "size": "1"}
            r = requests.post(url, headers=headers, data=data, timeout=20)
            r.encoding = "utf-8"
            r.raise_for_status()
            return r.json().get("total", 0)
        return retry_request(post)

    def get_page(page):
        def post():
            data = {"type": "search_contratos", "version": "108.0",
                    "query": f"adjudicanteid={adjudicante_id}", "page": str(page), "size": "25"}
            r = requests.post(url, headers=headers, data=data, timeout=20)
            r.encoding = "utf-8"
            r.raise_for_status()
            return r.json().get("items", [])
        return retry_request(post)

    try:
        total_contracts = get_total()
        page_count = math.ceil(total_contracts / 25)
        log_line(log_fp, f"🔍 NIF {nif} (ID {adjudicante_id}) — {total_contracts} contracts across {page_count} pages")

        contracts = []
        failed_pages = []

        for page in range(page_count):
            try:
                items = get_page(page)
                for c in items:
                    c["adjudicante_id"] = adjudicante_id
                    c["adjudicante_nif"] = nif
                    c["adjudicante_nome"] = name
                contracts.extend(items)
                log_line(log_fp, f"✅ Page {page} fetched with {len(items)} contracts.")
            except Exception as e:
                failed_pages.append(page)
                log_line(log_fp, f"❌ Page {page} failed: {e}")

            time.sleep(random.uniform(6, 8))  # Slower fetch

        retrieved = len(contracts)
        success_pages = page_count - len(failed_pages)

        # Save contracts
        if contracts:
            df_out = pd.DataFrame(contracts)
            excel_path = output_dir / f"{nif}_contracts.xlsx"
            df_out.to_excel(excel_path, index=False)
            log_line(log_fp, f"📁 Saved {retrieved} contracts → {excel_path}")
        else:
            log_line(log_fp, f"⚠️ No contracts retrieved.")

        # Save failed pages
        if failed_pages:
            with open(failed_fp, "w", encoding="utf-8") as f:
                json.dump(failed_pages, f)
            log_line(log_fp, f"⚠️ Failed pages saved to {failed_fp}")

        # Final summary
        log_line(log_fp, f"📊 Summary: Pages: {page_count}, Succeeded: {success_pages}, Failed: {len(failed_pages)}, Contracts: {retrieved}/{total_contracts}")

        # Append to completed
        with open(completed_log_path, "a") as f:
            f.write(nif + "\n")

    except Exception as e:
        log_line(log_fp, f"🚨 Critical error for NIF {nif}: {e}")

# 🏁 Launch NIFs sequentially, max 2 at once
if __name__ == "__main__":
    # Replace with your actual dataframe
    df = pd.DataFrame([
        {"nif": "506215695", "description": "Município do Fundão", "id": 1129},
        {"nif": "501143530", "description": "Município de Castelo Branco", "id": 3477},
        {"nif": "500051173", "description": "Município de Lisboa", "id": 1285},
        # Add more here
    ])

    # Limit to 2 threads max, sequential inside
    chunks = [row for _, row in df.iterrows()]
    with ThreadPoolExecutor(max_workers=2) as executor:
        executor.map(download_contracts_for_nif, chunks)


⏭ Skipping NIF 506215695, already processed.⏭ Skipping NIF 501143530, already processed.

🔍 NIF 500051173 (ID 1285) — 0 contracts across 0 pages
⚠️ No contracts retrieved.
📊 Summary: Pages: 0, Succeeded: 0, Failed: 0, Contracts: 0/0
